# RBC Segmentation

This script is designed to perform advanced image segmentation on whole microscope slide images, specifically targeting red blood cells (RBCs). The primary objectives of the script include:

**1. Segmentation of RBCs:** Utilizing image processing techniques, the script identifies and segments RBCs from the complex background of microscope slide images. This is achieved through various algorithms that enhance the visibility of cellular structures.

**2. Background Removal:** Once the RBCs are segmented, the script effectively removes the background, isolating the cells for further analysis. This step is crucial for improving the accuracy of subsequent image processing tasks.

**3. Edge Detection:** To enhance the features of the segmented RBCs, the script implements the **Sobel** edge detection algorithm. This technique highlights the edges of the cells, making it easier to analyze their morphology and structure.

In [1]:
#import libraries
import cv2
import numpy as np
from skimage import measure
import os

In [6]:
# Load the image
image = cv2.imread('1701151546-0007-R.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding to segment the red blood cells
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Find the contours of the segmented red blood cells
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a directory to save the individual red blood cell images
if not os.path.exists('blood2'):
    os.makedirs('blood2')

# Save each red blood cell as a separate image file
for i, contour in enumerate(contours):
    x, y, w, h = cv2.boundingRect(contour)
    cell_image = image[y:y+h, x:x+w]
    cv2.imwrite(f'blood2/cell2_{i+1}.png', cell_image)

print(f"{len(contours)} red blood cell images saved.")

924 red blood cell images saved.


In [4]:
# Load the image
image = cv2.imread('1704282807-0005-R.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding to segment the red blood cells
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Find the contours of the segmented red blood cells
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a directory to save the individual red blood cell images
if not os.path.exists('red'):
    os.makedirs('red')

# Save each unique red blood cell as a separate image file
unique_cells = []
for i, contour in enumerate(contours):
    x, y, w, h = cv2.boundingRect(contour)
    cell_image = image[y:y+h, x:x+w]
    
    # Resize the cell image to a fixed size
    cell_image = cv2.resize(cell_image, (32, 32))
    
    # Check if the cell is unique by comparing it to the existing ones
    is_unique = True
    for existing_cell in unique_cells:
        if cv2.absdiff(existing_cell, cell_image).sum() < 100:
            is_unique = False
            break
    
    if is_unique:
        cv2.imwrite(f'red/cell_{len(unique_cells)+1}.jpg', cell_image)
        unique_cells.append(cell_image)

print(f"{len(unique_cells)} unique red blood cell images saved.")


1917 unique red blood cell images saved.


## A  whole directory

In [8]:
import cv2
import os

# Input and output directories
input_dir = './test'  # Replace with the directory containing input images
output_dir = './result'  # Replace with the directory to save segmented images

# Create the output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process each image file in the input directory
image_files = os.listdir(input_dir)
for image_file in image_files:
    # Load the image
    image_path = os.path.join(input_dir, image_file)
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to segment the red blood cells
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find the contours of the segmented red blood cells
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a directory for the current image if it does not exist
    image_output_dir = os.path.join(output_dir, image_file.split('.')[0])  # Create a subdirectory with the image file name
    if not os.path.exists(image_output_dir):
        os.makedirs(image_output_dir)

    # Save each red blood cell as a separate image file
    for i, contour in enumerate(contours):
        x, y, w, h = cv2.boundingRect(contour)
        cell_image = image[y:y+h, x:x+w]
        cell_filename = f'cell_{i+1}.png'
        cell_path = os.path.join(image_output_dir, cell_filename)
        cv2.imwrite(cell_path, cell_image)

    print(f"{len(contours)} red blood cell images saved from {image_file}.")

14764 red blood cell images saved from 1409191647-0004-R.jpg.
3459 red blood cell images saved from 1704282807-0005-R.jpg.
9761 red blood cell images saved from 1305121398-0001-R_S.jpg.
924 red blood cell images saved from 1701151546-0007-R.jpg.


## Remove background

In [25]:
import cv2
import os
import numpy as np

# Input and output directories
input_dir = './sss'  # Replace with the directory containing input images
output_dir = './e3'  # Replace with the directory to save segmented images


# Create the output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process each image file in the input directory
image_files = os.listdir(input_dir)
for idx, image_file in enumerate(image_files, start=1):
    # Load the image
    image_path = os.path.join(input_dir, image_file)
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to segment the red blood cells
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find the contours of the segmented red blood cells
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a black background image
    black_bg = np.zeros_like(image)

    # Save each red blood cell as a separate image file with a black background
    for i, contour in enumerate(contours, start=1):
        x, y, w, h = cv2.boundingRect(contour)
        cell_image = image[y:y+h, x:x+w]
        cell_mask = np.zeros_like(image)
        cv2.drawContours(cell_mask, [contour], 0, (255, 255, 255), thickness=cv2.FILLED)
        cell_mask = cv2.resize(cell_mask, (cell_image.shape[1], cell_image.shape[0]))
        cell_image = cv2.bitwise_and(cell_image, cell_mask)
        cell_filename = f'imagemdx_{idx}_{i:02d}.png'  # Update the filename format
        cell_path = os.path.join(output_dir, cell_filename)
        cv2.imwrite(cell_path, cell_image)

    print(f"{len(contours)} red blood cell images saved from {image_file}.")

2 red blood cell images saved from imaged1748_953.png.
68 red blood cell images saved from imaged1521_513.png.
1 red blood cell images saved from imaged1661_883.png.
45 red blood cell images saved from imaged1675_897.png.
2 red blood cell images saved from imaged1602_800.png.


## Sobel edge and background removal

In [4]:
import cv2
import os
import numpy as np

# Set the input and output directories
input_dir = './sobel_input'
output_dir = './sobel_output'

# Create the output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process each image file in the input directory
image_files = os.listdir(input_dir)
for idx, image_file in enumerate(image_files, start=1):
    # Load the image
    image_path = os.path.join(input_dir, image_file)
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Sobel edge detection
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    sobel = np.sqrt(sobelx**2 + sobely**2)
    sobel = np.uint8(sobel)

    # Apply thresholding to segment the red blood cells
    _, thresh = cv2.threshold(sobel, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find the contours of the segmented red blood cells
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a black background image
    black_bg = np.zeros_like(image)

    # Save each red blood cell as a separate image file with a black background
    for i, contour in enumerate(contours, start=1):
        x, y, w, h = cv2.boundingRect(contour)
        cell_image = image[y:y+h, x:x+w]
        cell_mask = np.zeros_like(image)
        cv2.drawContours(cell_mask, [contour], 0, (255, 255, 255), thickness=cv2.FILLED)
        cell_mask = cv2.resize(cell_mask, (cell_image.shape[1], cell_image.shape[0]))
        cell_image = cv2.bitwise_and(cell_image, cell_mask)
        cell_filename = f'image_{idx}_{i:02d}.png'
        cell_path = os.path.join(output_dir, cell_filename)
        cv2.imwrite(cell_path, cell_image)

    print(f"{len(contours)} red blood cell images saved from {image_file}.")


6 red blood cell images saved from image6_01.png.
2 red blood cell images saved from image39_10.png.
1 red blood cell images saved from image1_03.png.
1 red blood cell images saved from image9_01.png.
1 red blood cell images saved from image12_01.png.
2 red blood cell images saved from image27_01.png.
1 red blood cell images saved from image22_01.png.
1 red blood cell images saved from image21_01.png.
10 red blood cell images saved from image19_18.png.
2 red blood cell images saved from image4_15.png.
1 red blood cell images saved from image26_25.png.
1 red blood cell images saved from image40_02.png.
3 red blood cell images saved from image38_10.png.
1 red blood cell images saved from image36_01.png.
1 red blood cell images saved from image31_02.png.
1 red blood cell images saved from image42_03.png.
1 red blood cell images saved from image5_01.png.
1 red blood cell images saved from image8_03.png.
1 red blood cell images saved from image20_02.png.
1 red blood cell images saved from i